In [1]:
# hello world
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
import string
import re
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Word2Vec
sc = pyspark.SparkContext()
from pyspark.sql.functions import concat_ws, array
from pyspark.sql import *
import numpy
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

spark = SparkSession.builder.appName("Projec3")\
        .config("spark.some.config.option","some-value")\
        .getOrCreate()

df = spark.read.load("/home/cse587/CSE587/Project3/train.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')

stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']

#punc = string.punctuation
def removePunctuation(s):
    s = ' '.join([i for i in s.split() if i not in stopwords])
    return re.sub(r'[^\w\s]',"",s)
#data = re.sub(r'[^\w\s]',"",data)

udf = UserDefinedFunction(lambda x: removePunctuation(x), StringType())
df = df.withColumn('plot', udf(df.plot))
#new_df = df.withColumn("plot",)



genreMap = spark.read.load("/home/cse587/CSE587/Project3/mapping.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')
genreMap = genreMap.withColumnRenamed('_c0','id')
genreMap = genreMap.withColumnRenamed('0','genre')

#this converts a column into a list for use inside encodeGenre() 
#since you cant perform computations on a dataframe inside a UDF
genreList = genreMap.select("genre").rdd.flatMap(lambda x: x).collect()


def encodeGenre(s):
    #s : the value of col "genre" at some row
    retVal=""
    #s = re.sub(r'[^\\'']',"",s)
    #print("s: ",s)
    #print("type of s: ",type(s))
    if(s == None):
        return '00000000000000000000'
    #convert from string to list
    s = s[1:-1].split(", ")
    #print("type of s: ",type(s))
    #print(s)
    
    #remove single quotes, regex wasnt working for some reason
    temp=[]
    for x in s:
        #s.remove(x)
        x=x[1:-1]
#         print(x)
#         index = genreMap["0"==x]
#         print(index)
        temp.append(x)
    s=temp
    #print(s)
    
    #create one hot encoding, 
    for genre in genreList:
        if genre in s:
            retVal=retVal+"1"
        else:
            retVal=retVal+"0"
##### apparetnly it aint kosher to perform df operations inside a udf... so this hour of work was for nothing
#     t = genreMap.withColumn("encoding",F.when(genreMap.genre.isin(s),1).otherwise(0))
#     #t.show()
#     retVal = t.select("encoding").rdd.flatMap(lambda x: x).collect()
#     retVal = ''.join(map(str,retVal))
    return retVal
    
    

udf2 = UserDefinedFunction(lambda x: encodeGenre(x), StringType())
df = df.withColumn('genre',udf2(df.genre))

#install numpy if you havent already, otherwise one of the ml.featuer imports will throw an error cause it needs numpy
#%pip install numpy

In [2]:
mylist = df.collect()


In [3]:
print(mylist[0]['plot'])

Shlykov hardworking taxi driver Lyosha saxophonist develop bizarre lovehate relationship despite prejudices realize arent different all


In [4]:
#part 1 code
#taken from https://github.com/apache/spark/blob/master/examples/src/main/python/ml/tf_idf_example.py
tokenizer = Tokenizer(inputCol="plot", outputCol="words")
wordsData = tokenizer.transform(df)

# hashingTF = HashingTF(inputCol="words", outputCol="hashFeatures")
# featurizedData = hashingTF.transform(wordsData)

# #this will do part 2
# idf = IDF(inputCol="hashFeatures", outputCol="idf_output")
# idfModel = idf.fit(featurizedData)
# featurizedData = idfModel.transform(featurizedData)

word2Vec = Word2Vec(inputCol='words',outputCol='features')
w2vmodel = word2Vec.fit(wordsData)
featurizedData = w2vmodel.transform(wordsData)


# assembler = VectorAssembler(inputCols=['vectors','idf_output'],outputCol = 'features')
# featurizedData= assembler.transform(featurizedData)
print('done')

done


In [5]:
synonyms = w2vmodel.findSynonyms('jester', 5)
synonyms.show(5)

+--------+------------------+
|    word|        similarity|
+--------+------------------+
|  monami|0.5761631727218628|
|sorcerer|0.5508697032928467|
|anaconda|0.5376604795455933|
|disarmed|0.5315737128257751|
|tunneler|0.5255194902420044|
+--------+------------------+



In [6]:
df_test = spark.read.load("/home/cse587/CSE587/Project3/test.csv",sep=",",format="csv",inferSchema = True,header = True,escape ='"')
df_test = df_test.withColumn('plot',udf(df_test.plot))

tokenizer2 = Tokenizer(inputCol="plot", outputCol="words")
test_cases = tokenizer2.transform(df_test)

# hashingTF2 = HashingTF(inputCol="words", outputCol="hashFeatures")
# test_cases = hashingTF2.transform(wordsData2)
# idfModel_testing = idf.fit(test_cases)
# test_cases = idfModel_testing.transform(test_cases)

word2Vec2 = Word2Vec(inputCol='words',outputCol='features')
w2vmodel2 = word2Vec2.fit(test_cases)
test_cases = w2vmodel2.transform(test_cases)

# assembler2 = VectorAssembler(inputCols=['hashFeatures','idf_output'],outputCol = 'features')
# test_cases= assembler2.transform(test_cases)

In [7]:
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import substring 
from pyspark.sql.types import *

all_label_set = featurizedData.drop('movie_id', 'movie_name', 'plot', 'words')

columns = all_label_set.columns
columns = columns[::-1]
all_label_set = all_label_set[columns]
all_label_set = all_label_set.withColumnRenamed('genre','all_label')

In [8]:
for i in range(20):
    #substr(start = 0,len)
    x = all_label_set.withColumn("label", all_label_set.all_label.substr(i+1,1).cast(IntegerType()))
    x = x.drop('all_label')
    lr = LogisticRegression(maxIter = 10,regParam=0.0,elasticNetParam=0.0)
    lr_model =lr.fit(x)

    #for all data
    #predict
    print(i)
    test_cases = lr_model.transform(test_cases)
    test_cases = test_cases.withColumn(genreList[i],test_cases['prediction'].cast(IntegerType()))
    test_cases = test_cases.drop('prediction','probability','rawPrediction')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
#test_cases.filter(test_cases['Drama'] != test_cases['Thriller']).show()
#test_cases = test_cases.withColumn('test', test_cases.movie_id.substr(1,2))
#test_cases.select('movie_id','test').show()
#test_cases = test_cases.drop('test')

In [9]:
test_cases = test_cases.drop('movie_name', 'plot', 'words','features','rawFeatures')
test_cases =test_cases.withColumn('all_genre',concat_ws(' ',array([genreList[i] for i in range(20)])))
for i in range(20):
    test_cases = test_cases.drop(genreList[i])

In [10]:
'''
for i in range(20):
    a = test_cases.filter(test_cases[genreList[i]] != 0).filter(test_cases[genreList[i]] != 1)
    a.select('movie_id',genreList[i]).show(vertical=True)
'''
#test_cases.filter(test_cases['movie_id'] < 10).show(vertical=True)

#test_cases.show(vertical=True)

"\nfor i in range(20):\n    a = test_cases.filter(test_cases[genreList[i]] != 0).filter(test_cases[genreList[i]] != 1)\n    a.select('movie_id',genreList[i]).show(vertical=True)\n"

In [12]:
test_cases.write.option("delimiter", ',').csv('./part3_take3', escapeQuotes=False)